In [15]:
# this notebook creates and populates tables for UN Data reports

In [2]:
from functions import *
from SQL_functions import *
from stats_forumulas import *
import config
import mysql.connector
from mysql.connector import errorcode
import math
import pandas as pd
import numpy as np
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [6]:
cur, cnx = connect_to_db('gender')

## Creating Tables for UN Data

In [1]:
TABLES = {}
csv_names = ['child_birth_rate', 'contraceptive_prevalence', 'gender_urban_ratio', 
             'gender_rural_ratio', 'ter_ed_enrollment', 'women_teach_sec', 'women_teach_ter', 
             'abortion_laws', 'abortion_rate', 'abortion_info']

In [ ]:
for name in csv_names:
    TABLES[name] = (f"""
    CREATE TABLE {name} (
    country varchar(50),
    subgroup varchar(200),
    year int(50),
    unit varchar(50),
    value int(50),
    PRIMARY KEY (country, subgroup, year)
    ) 
    """)

In [7]:
# added to DB
for name in csv_names:
    cur.execute(TABLES[name])

In [8]:
# close cursor
cnx.close()
cur.close() # returned True

True

## Populating our Tables

In [14]:
# # see SQL_functions.py for function and helper function definitions
for name in csv_names:
    if name != 'abortion_info':
        insert_data(name, name)
    else:
        insert_data_special(name, name)

/Users/rebeccarosen/Desktop/flatiron 2019/mod3/project2/gender-and-tech/SQL_functions.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df2['Year'] = pd.to_numeric(df2['Year'], errors = 'coerce')
/Users/rebeccarosen/Desktop/flatiron 2019/mod3/project2/gender-and-tech/SQL_functions.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df2['Value'] = pd.to_numeric(df2['Value'], errors = 'coerce')


In [21]:
cur, cnx = connect_to_db('gender')
df = pd.read_csv('Data To Use/Honeypot_Women_in_Tech.csv')
cur.executemany(insert_more_query('women_in_tech'), get_more_tuples(df))
cnx.commit()
cnx.close()